# Model-Based Reinforcement Learning


In [19]:
# Install environment and visualization dependencies
!pip install highway-env

# Environment
import gymnasium as gym
import highway_env

gym.register_envs(highway_env)

# Models and computation
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import namedtuple

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline

We also define a simple helper function for visualization of episodes:

In [20]:
import sys
from tqdm.notebook import trange
!pip install tensorboardx gym pyvirtualdisplay
!apt-get install -y xvfb ffmpeg
!git clone https://github.com/gupann/CS269-Parking.git 2> /dev/null
sys.path.insert(0, '/content/HighwayEnv/scripts/')
from utils import record_videos, show_videos

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.16).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


### Let's try it!

Make the environment, and run an episode with random actions:

In [17]:
env = gym.make("parking-v0", render_mode="rgb_array")
env = record_videos(env)
env.reset()
done = False
while not done:
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
env.close()
show_videos()

/usr/local/lib/python3.12/dist-packages/gymnasium/wrappers/rendering.py:293: UserWarning: WARN: Overwriting existing videos at /content/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/usr/local/lib/python3.12/dist-packages/gymnasium/wrappers/rendering.py:434: UserWarning: WARN: Unable to save last video! Did you call close()?
  logger.warn("Unable to save last video! Did you call close()?")


The environment is a `GoalEnv`, which means the agent receives a dictionary containing both the current `observation` and the `desired_goal` that conditions its policy.

In [18]:
print("Observation format:", obs)

Observation format: OrderedDict({'observation': array([ 0.20859307, -0.18080901,  0.97863928, -0.74214927,  0.79679583,
       -0.60424863]), 'achieved_goal': array([ 0.20859307, -0.18080901,  0.97863928, -0.74214927,  0.79679583,
       -0.60424863]), 'desired_goal': array([2.600000e-01, 1.400000e-01, 0.000000e+00, 0.000000e+00,
       6.123234e-17, 1.000000e+00])})


There is also an `achieved_goal` that won't be useful here (it only serves when the state and goal spaces are different, as a projection from the observation to the goal space).

Alright! We are now ready to apply the model-based reinforcement learning paradigm.